In [73]:
import time
from tinkerforge.ip_connection import IPConnection, Error
from tinkerforge.bricklet_performance_dc import BrickletPerformanceDC
from tinkerforge.bricklet_silent_stepper_v2 import BrickletSilentStepperV2

%matplotlib inline

HOST = "localhost"
PORT = 4223
PDC_UID = "SSV" 
SS_UID = "2akW"

# Create IP connection
ipcon = IPConnection()

# Create device objects
ss1 = BrickletSilentStepperV2(SS_UID, ipcon)
pdc = BrickletPerformanceDC(PDC_UID, ipcon)
SS_SPEED = 500

try:
    ipcon.connect(HOST, PORT)
    print("Connected to brickd")
except Error as e:
    print(f"Error connecting to brickd: {e}")


Connected to brickd


In [74]:
def initialize_motor(ss):
    # 416 mA, for 10W 24V system
    ss.set_motor_current(416) 
    # 1/8 steps (interpolated)
    ss.set_step_configuration(ss1.STEP_RESOLUTION_8, True) 
    ss.set_max_velocity(SS_SPEED) 
    ss.set_speed_ramping(SS_SPEED * 100, SS_SPEED * 100)
    ss.set_enabled(True) 

def turn_on_pump():
    pdc.set_drive_mode(pdc.DRIVE_MODE_DRIVE_COAST)  
    pdc.set_motion(32767, 32767)  # Optional: set max accel/decel just in case
    pdc.set_velocity(12000)       # 7K gives a nice gentle spray
    pdc.set_enabled(True)         # Enable motor

def turn_off_pump():
    pdc.set_velocity(0)           # Stop motor
    pdc.set_enabled(False)        # Cut power — acts like a relay off



In [ ]:
import time
from datetime import datetime
from tinkerforge.ip_connection import IPConnection, Error, TimeoutException
from tinkerforge.bricklet_performance_dc import BrickletPerformanceDC
from tinkerforge.bricklet_silent_stepper_v2 import BrickletSilentStepperV2

HOST = "localhost"
PORT = 4223
PDC_UID = "SSV"
SS_UID = "2akW"
SS_SPEED = 500

# Global connection and devices
ipcon = IPConnection()
pdc = None
ss1 = None

# ----- Safe Connection to brickd -----
def connect_to_brickd():
    global ipcon
    while True:
        try:
            ipcon.connect(HOST, PORT)
            print("Connected to brickd")
            break
        except Error as e:
            print(f"Error connecting to brickd: {e}. Retrying in 5 seconds...")
            time.sleep(5)

# ----- Safe Tinkerforge call wrapper -----
def safe_call(func, *args, retries=3, delay=1):
    for attempt in range(retries):
        try:
            return func(*args)
        except TimeoutException as e:
            print(f"Timeout in {func.__name__}, attempt {attempt+1}: {e}")
        except Error as e:
            print(f"Tinkerforge error in {func.__name__}: {e}")
        time.sleep(delay)
    raise RuntimeError(f"{func.__name__} failed after {retries} retries.")

# ----- Motor initialization -----
def initialize_motor(ss):
    safe_call(ss.set_motor_current, 416)
    safe_call(ss.set_step_configuration, ss.STEP_RESOLUTION_8, True)
    safe_call(ss.set_max_velocity, SS_SPEED)
    safe_call(ss.set_speed_ramping, SS_SPEED * 100, SS_SPEED * 100)
    safe_call(ss.set_enabled, True)

# ----- Pump control -----
def turn_on_pump():
    safe_call(pdc.set_drive_mode, pdc.DRIVE_MODE_DRIVE_COAST)
    safe_call(pdc.set_motion, 32767, 32767)
    safe_call(pdc.set_velocity, 12000)
    safe_call(pdc.set_enabled, True)

def turn_off_pump():
    safe_call(pdc.set_velocity, 0)
    safe_call(pdc.set_enabled, False)

# ----- Main Experiment Loop -----
def start_experiment(duration=14 * 24 * 60 * 60, interval=15 * 60, misting_time=4):
    global pdc, ss1

    connect_to_brickd()

    # Create device objects after connection
    ss1 = BrickletSilentStepperV2(SS_UID, ipcon)
    pdc = BrickletPerformanceDC(PDC_UID, ipcon)

    safe_call(initialize_motor, ss1)
    safe_call(ss1.set_steps, 100000)  # Positioning stepper

    start_time = time.time()
    misting_iteration = 0

    try:
        while time.time() - start_time < duration:
            try:
                print(f'Misting iteration: {misting_iteration} at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
                turn_on_pump()
                time.sleep(misting_time)
                turn_off_pump()
                print('Misting complete.\n')
            except (TimeoutException, Error) as e:
                print(f"Device error during misting: {e}. Reconnecting...")
                try:
                    ipcon.disconnect()
                except:
                    pass
                connect_to_brickd()
                pdc = BrickletPerformanceDC(PDC_UID, ipcon)
                ss1 = BrickletSilentStepperV2(SS_UID, ipcon)
                initialize_motor(ss1)

            misting_iteration += 1
            time.sleep(interval)

    except KeyboardInterrupt:
        print("Experiment interrupted by user.")

    finally:
        print("Shutting down safely...")
        try:
            turn_off_pump()
            safe_call(ss1.set_enabled, False)
        except:
            print("Failed to shut down motors cleanly.")
        try:
            ipcon.disconnect()
        except:
            pass
        print("Experiment ended.")

# Uncomment to run directly


In [ ]:
# Every 6 spurts (for 2 seconds each) is 30 mL
# so each spurt is 5mL
start_experiment(duration=14 * 24 * 60 * 60, interval=10 * 60, misting_time=10)


Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:57:49
starting misting!
stopping misting!
Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:57:58
starting misting!
stopping misting!
Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:58:07
starting misting!
stopping misting!


KeyboardInterrupt: 

In [ ]:
# Disconnect from brickd
ipcon.disconnect()
